In [4]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [5]:
dataset = pd.read_csv('SPY_data_to57.csv',index_col="date",parse_dates=True)
dataset = dataset.dropna(axis=0, how='all')  #get rid of rows with NaN

In [6]:
# dataset.head()
# dataset.isna().any()
# dataset.info()
# type(dataset["4. close"])

In [7]:
# dataset['1. open'].plot(figsize=(16,6))

In [8]:
# 7 day rolling mean
#dataset.rolling(7).mean().head(20)

In [9]:
# dataset['1. open'].plot(figsize=(16,6))
# dataset.rolling(window=30).mean()['4. close'].plot()

In [10]:
# dataset['Close: 30 Day Mean'] = dataset['4. close'].rolling(window=30).mean()
# dataset[['4. close','Close: 30 Day Mean']].plot(figsize=(16,6))

In [11]:
# Optional specify a minimum number of periods
# dataset['4. close'].expanding(min_periods=1).mean().plot(figsize=(16,6))

In [12]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))

In [13]:
def get_train_sets(col_name, sws):
    training_set=dataset[col_name]
    #training_set=dataset
    training_set=pd.DataFrame(training_set)
    #print(training_set.head())
    # Feature Scaling
    training_set_scaled = sc.fit_transform(training_set)
    #print(len(training_set_scaled))
    #Empty Lists. Populated below with the array elements of the scaled training set#Convert 
    X_train = []
    y_train = []
    for i in range(4000, 5348):
        X_train.append(training_set_scaled[i-sws:i, 0])
        y_train.append(training_set_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    #X_train.shape
    #y_train.shape
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    #X_train.shape
    return X_train, y_train

In [14]:
# Importing the Keras libraries and packages
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM

In [12]:
import pickle

def train_model(name, X_train, y_train):
    regressor = Sequential()
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(LSTM(units = 50))
    regressor.add(Dense(units = 1))
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

    #history = regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_split = 0.2)
    history = regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)
    regressor.save(name+'.h5')
    with open(name, 'wb') as f:
        pickle.dump(history, f)
    return regressor, history

In [13]:
import matplotlib.pyplot as plt

def plot_results(history):
    plt.plot(history.history['acc'], '-b', label = 'acc')
    plt.plot(history.history['val_acc'], '-r', label = 'val_acc')
    plt.legend(loc='upper left')
    plt.title('Model Accuracy')
    plt.ylabel('Acc')
    plt.xlabel('Epoch')
    plt.show()

    plt.clf()

    plt.plot(history.history['loss'], '-b', label = 'loss')
    plt.plot(history.history['val_loss'], '-r', label = 'val_loss')
    plt.legend(loc='upper left')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()

In [14]:
# Making the predictions and visualising the results
from sklearn.metrics import mean_squared_error
import math

In [15]:
def get_X_test_all(col_name, dataset, sws):
    training_set=dataset[col_name]
    training_set=pd.DataFrame(training_set)
    training_set_scaled = sc.fit_transform(training_set)
    X_test_all = []
    y_test_all = []
    for i in range(sws, 5356):
        X_test_all.append(training_set_scaled[i-sws:i, 0])
        y_test_all.append(dataset[col_name][i])
    X_test_all = np.array(X_test_all)
    X_test_all = np.reshape(X_test_all, (X_test_all.shape[0], X_test_all.shape[1], 1))
    return X_test_all, y_test_all

In [16]:
def plot_preds_all(regressor, X_test, y_test, y_type):
    predicted_stock_price = regressor.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    predicted_stock_price=pd.DataFrame(predicted_stock_price)

    plt.plot(y_test, color = 'red', label = 'Real SPY')
    plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted SPY')
    plt.title(y_type + ' All Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()
    testScore = math.sqrt(mean_squared_error(y_test, predicted_stock_price))
    print(y_type + ' All Stocks Test Score: %.2f RMSE' % (testScore))

In [17]:
def get_X_test_48(col_name, dataset, sws):
    training_set=dataset[col_name]
    training_set=pd.DataFrame(training_set)
    training_set_scaled = sc.fit_transform(training_set)
    X_test_48 = []
    y_test_48 = []
    for i in range(5349, 5356):
        X_test_48.append(training_set_scaled[i-sws:i, 0])
        y_test_48.append(dataset[col_name][i])
    X_test_48 = np.array(X_test_48)
    X_test_48 = np.reshape(X_test_48, (X_test_48.shape[0], X_test_48.shape[1], 1))
    return X_test_48, y_test_48

In [18]:
def plot_preds_48(regressor, X_test, y_test, y_type):
    predicted_stock_price = regressor.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    predicted_stock_price=pd.DataFrame(predicted_stock_price)

    dates_data = pd.read_csv('SPY_data_to57.csv',usecols=["date"])
    #print(dates_data.columns.values)
    x = dates_data['date'][5349:5356]
    plt.plot(x, y_test, color = 'red', label = 'Real SPY')
    plt.plot(x, predicted_stock_price, color = 'blue', label = 'Predicted SPY')
    plt.title(y_type + ' Stock Price Prediction 4/8 to 4/16')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.xticks(rotation='vertical')
    plt.legend()
    plt.show()
    testScore = math.sqrt(mean_squared_error(y_test, predicted_stock_price))
    print(y_type + ' 4/8 to 4/16 Test Score: %.2f RMSE' % (testScore))

In [19]:
"""
We will fill in 5/8 to 5/14 in SPY_58_to_now.csv. Right now this has dummy values for those days but once the actual values 
come out we'll fill them in. The notebook will predict values from 5/8 to 5/14. It can also predict just 1 value by taking a 
subset of these predictions.  It can also predict 5/15 or another future day by changing one line of code  
(see the comment in get_X_test_58_to_now). 
"""
dataset_test = pd.read_csv('SPY_58_to_now.csv',index_col="date",parse_dates=True)
dataset_test = dataset_test.dropna(axis=0, how='all')  #get rid of rows with NaN

def get_X_test_417(col_name, dataset, dataset_test, sws, k):
    real_stock_price = dataset_test.iloc[:, k:k+1].values
    dataset_total = pd.concat((dataset[col_name], dataset_test[col_name]), axis = 0)
    inputs = dataset_total[len(dataset_total) - len(dataset_test) - sws:].values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    X_test_417 = []
    for i in range(sws, sws+len(dataset_test)):
        X_test_417.append(inputs[i-sws:i, 0])
    #X_test
    X_test_417 = np.array(X_test_417)
    #X_test_417.shape
    X_test_417 = np.reshape(X_test_417, (X_test_417.shape[0], X_test_417.shape[1], 1))
    #X_test_417.shape
    return X_test_417, real_stock_price

In [20]:
def plot_preds_417(regressor, X_test, real_stock_price, y_type):
    predicted_stock_price = regressor.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    predicted_stock_price=pd.DataFrame(predicted_stock_price)
    #predicted_stock_price.info()
    plt.plot(real_stock_price, color = 'red', label = 'Real SPY')
    plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted SPY')
    plt.title(y_type + ' Stock Price Prediction 4/17 - 5/3')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()
    testScore = math.sqrt(mean_squared_error(real_stock_price, predicted_stock_price))
    print(y_type + ' 4/17 - 5/3 Test Score: %.2f RMSE' % (testScore))

In [21]:
def get_X_test_58_to_now(col_name, dataset, dataset_test, sws, k):
#     training_set=dataset[col_name]
#     training_set=pd.DataFrame(training_set)
#     training_set_scaled = sc.fit_transform(training_set)
    
    real_stock_price = dataset_test.iloc[:, k:k+1].values
    dataset_total = pd.concat((dataset[col_name], dataset_test[col_name]), axis = 0)
    inputs = dataset_total[len(dataset_total) - len(dataset_test) - sws:].values  #sws + len(dataset_test) values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    X_test = []
    for i in range(sws, sws+len(dataset_test)):  #+15 is 4/17 to 5/8. use sws+len(dataset_test)+1 if want 5/15 (not in csv)
        X_test.append(inputs[i-sws:i, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    return X_test, real_stock_price

In [22]:
def predict_58_to_now(regressor, X_test, real_stock_price, y_type, col_name):
#     training_set=dataset[col_name]
#     training_set=pd.DataFrame(training_set)
#     training_set_scaled = sc.fit_transform(training_set)
    
    predicted_stock_price = regressor.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    predicted_stock_price=pd.DataFrame(predicted_stock_price)
    #testScore = math.sqrt(mean_squared_error(real_stock_price[14:], predicted_stock_price))
    
    base = datetime.datetime.strptime('May 8 2019', '%b %d %Y')
    date_list = [base + datetime.timedelta(days=x) for x in range(0, 5)]
    date_dict = {i:date.date() for i, date in enumerate(date_list)}
    
#     plt.plot(date_list, real_stock_price[14:], color = 'red', label = 'Real SPY')
#     plt.plot(date_list, predicted_stock_price, color = 'blue', label = 'Predicted SPY')
#     plt.title(y_type + ' Stock Price Prediction 5/8 - 5/12')
#     plt.xticks(rotation='vertical')
#     plt.ylabel('Stock Price')
#     plt.legend()
    
#     predicted_stock_price.columns = ['Predictions']
#     predicted_stock_price.rename(index=date_dict, inplace=True)
#     predicted_stock_price['Actual'] = real_stock_price[14:]
    
    print(predicted_stock_price)
#     plt.show()
#     print(y_type + ' 5/8 - 5/122 Test Score: %.2f RMSE' % (testScore))
    return predicted_stock_price

In [23]:
def predict_next(col_name, dataset, regressor):
    dataset = pd.read_csv('SPY_data.csv',index_col="date",parse_dates=True)
    dataset = dataset.dropna(axis=0, how='all')  #get rid of rows with NaN
    inputs = dataset[col_name].values.reshape(-1,1)
    inputs = sc.transform(inputs)
    X_test = []
    X_test.append(inputs[len(dataset)-sws:len(dataset)+1, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predicted_stock_price = regressor.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    predicted_stock_price = pd.DataFrame(predicted_stock_price)
    print("%.2f" % predicted_stock_price[0])
    return predicted_stock_price

In [15]:
###### OPEN ######
sws = 100
X_train_open, y_train_open = get_train_sets('1. open', sws)

#regressor_open, history = train_model('SPY_LSTM_4000_5348_sws100_open', X_train_open, y_train_open)
regressor_open = load_model('SPY_LSTM_4000_5348_sws100_open.h5')

In [18]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train_open.shape[1], 1)))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(LSTM(units = 50))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])
regressor.summary()

In [25]:
# #history = pickle.load(open("SPY_LSTM_4000_5348_sws100_open", "rb" ))
# #plot_results(history)

# X_test_all, y_test_all = get_X_test_all('1. open', dataset, sws)
# plot_preds_all(regressor_open, X_test_all, y_test_all, "Open")

# X_test_48, y_test_48 = get_X_test_48('1. open', dataset, sws)
# plot_preds_48(regressor_open, X_test_48, y_test_48, "Open")

# X_test_417, real_stock_price = get_X_test_417('1. open', dataset, dataset_test, sws, 0)
# plot_preds_417(regressor_open, X_test_417, real_stock_price, "Open")

In [26]:
# X_test_nextday, real_stock_price = get_X_test_58_to_now('1. open', dataset, dataset_test, sws, 0)
# open_preds = predict_58_to_now(regressor_open, X_test_nextday, real_stock_price, "Open", '1. open')

In [27]:
open_preds = predict_next('1. open', dataset, regressor_open)

288.27


In [28]:
###### HIGH ######
sws = 100
X_train_high, y_train_high = get_train_sets('2. high', sws)

#regressor_high, history = train_model('SPY_LSTM_4000_5348_sws100_high', X_train_high, y_train_high)
regressor_high = load_model('SPY_LSTM_4000_5348_sws100_high.h5')

In [29]:
# #history = pickle.load(open("SPY_LSTM_4000_5348_sws100_open", "rb" ))
# #plot_results(history)

# X_test_all, y_test_all = get_X_test_all('2. high', dataset, sws)
# plot_preds_all(regressor_high, X_test_all, y_test_all, "High")

# X_test_48, y_test_48 = get_X_test_48('2. high', dataset, sws)
# plot_preds_48(regressor_high, X_test_48, y_test_48, "High")

# X_test_417, real_stock_price = get_X_test_417('2. high', dataset, dataset_test, sws, 1)
# plot_preds_417(regressor_high, X_test_417, real_stock_price, "High")

In [30]:
# X_test_nextday, real_stock_price = get_X_test_58_to_now('2. high', dataset, dataset_test, sws, 1)
# high_preds = predict_58_to_now(regressor_high, X_test_nextday, real_stock_price, "High", '2. high')

In [31]:
high_preds = predict_next('2. high', dataset, regressor_high)

287.37


In [32]:
###### LOW ######
sws = 150
X_train_low, y_train_low = get_train_sets('3. low', sws)

#regressor_low, history = train_model('SPY_LSTM_4000_5348_sws150_low', X_train_low, y_train_low)
regressor_low = load_model('SPY_LSTM_4000_5348_sws150_low.h5')

In [33]:
# #history = pickle.load(open("SPY_LSTM_4000_5348_sws150_low", "rb" ))
# #plot_results(history)

# X_test_all, y_test_all = get_X_test_all('3. low', dataset, sws)
# plot_preds_all(regressor_low, X_test_all, y_test_all, "Low")

# X_test_48, y_test_48 = get_X_test_48('3. low', dataset, sws)
# plot_preds_48(regressor_low, X_test_48, y_test_48, "Low")

# X_test_417, real_stock_price = get_X_test_417('3. low', dataset, dataset_test, sws, 2)
# plot_preds_417(regressor_low, X_test_417, real_stock_price, "Low")

In [34]:
# X_test_nextday, real_stock_price = get_X_test_58_to_now('3. low', dataset, dataset_test, sws, 2)
# low_preds = predict_58_to_now(regressor_low, X_test_nextday, real_stock_price, "Low", '3. low')

In [35]:
low_preds = predict_next('3. low', dataset, regressor_low)

279.74


In [36]:
###### CLOSE ######
sws = 150
X_train_close, y_train_close = get_train_sets('4. close', sws)

#regressor_close, history = train_model('SPY_LSTM_4000_5348_sws150_close', X_train_close, y_train_close)
regressor_close = load_model('SPY_LSTM_4000_5348_sws150_close.h5')

In [37]:
# #history = pickle.load(open("SPY_LSTM_4000_5348_sws150_close", "rb" ))
# #plot_results(history)

# X_test_all, y_test_all = get_X_test_all('4. close', dataset, sws)
# plot_preds_all(regressor_close, X_test_all, y_test_all, "Close")

# X_test_48, y_test_48 = get_X_test_48('4. close', dataset, sws)
# plot_preds_48(regressor_close, X_test_48, y_test_48, "Close")

# X_test_417, real_stock_price = get_X_test_417('4. close', dataset, dataset_test, sws, 3)
# plot_preds_417(regressor_close, X_test_417, real_stock_price, "Close")

In [38]:
# X_test_nextday, real_stock_price = get_X_test_58_to_now('4. close', dataset, dataset_test, sws, 3)
# close_preds = predict_58_to_now(regressor_close, X_test_nextday, real_stock_price, "Close", '4. close')

In [39]:
close_preds = predict_next('4. close', dataset, regressor_close)

289.20


In [40]:
###### VOL ######
sws = 150
#X_train_vol, y_train_vol = get_train_sets('5. volume', sws)

#regressor_vol, history = train_model('SPY_LSTM_4000_5348_sws150_vol', X_train_vol, y_train_vol)
regressor_vol = load_model('SPY_LSTM_4000_5348_sws150_vol.h5')

In [41]:
# #history = pickle.load(open("SPY_LSTM_4000_5348_sws150_vol", "rb" ))
# #plot_results(history)

# X_test_all, y_test_all = get_X_test_all('5. volume', dataset, sws)
# plot_preds_all(regressor_vol, X_test_all, y_test_all, "Vol")

# X_test_48, y_test_48 = get_X_test_48('5. volume', dataset, sws)
# plot_preds_48(regressor_vol, X_test_48, y_test_48, "Vol")

# X_test_417, real_stock_price = get_X_test_417('5. volume', dataset, dataset_test, sws, 4)
# plot_preds_417(regressor_vol, X_test_417, real_stock_price, "Vol")

In [42]:
# X_test_nextday, real_stock_price = get_X_test_58_to_now('5. volume', dataset, dataset_test, sws, 4)
# predict_58_to_now(regressor_vol, X_test_nextday, real_stock_price, "Vol")

In [43]:
print("%.2f" % open_preds[0])
print("%.2f" % close_preds[0])
print("%.2f" % high_preds[0])
print("%.2f" % low_preds[0])

In [44]:
# IT BARELY MADE A DIFFERENCE when used 4/17 to 5/7 duplicate data in predictions, nor did it make a difference to fit
# sc transform on another column. What DID make a difference was using windows 150 vs 100 for low and close.
# sws 100 was actually better for everything!